In [ ]:
import os
import gc
import sys

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchinfo import summary

from kitti_data_load import KITTI
from prednet import PredNet

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
num_epochs = 150
batch_size = 1
A_channels = (3, 48, 96, 192)
R_channels = (3, 48, 96, 192)
lr = 0.001 # if epoch < 75 0.001 else 0.0001
nt = 10 # num of time steps

DATA_DIR = 'kitti_data_raw'

In [ ]:
layer_loss_weights = Variable(torch.FloatTensor([[1.], [0.], [0.], [0.]]).cuda())
time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
time_loss_weights[0] = 0
time_loss_weights = Variable(time_loss_weights.cuda())
print("layer loss: ", layer_loss_weights.shape, '\n',
      "time loss: ", time_loss_weights.shape)

In [ ]:
train_file = os.path.join(DATA_DIR, 'X_train.hkl')
train_sources = os.path.join(DATA_DIR, 'sources_train.hkl')
val_file = os.path.join(DATA_DIR, 'X_val.hkl')
val_sources = os.path.join(DATA_DIR, 'sources_val.hkl')

# time_step, height, width, channels
kitti_train = KITTI(train_file, train_sources, nt)
kitti_val = KITTI(val_file, val_sources, nt)

In [ ]:
train_loader = DataLoader(kitti_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(kitti_val, batch_size=batch_size, shuffle=True)

In [ ]:
model = PredNet(R_channels, A_channels, output_mode='error_all')
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()

In [ ]:
model.eval()

In [ ]:
summary(model, (batch_size,nt,3,128,160))

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
def lr_scheduler(optimizer, epoch):
    if epoch < num_epochs //2:
        return optimizer
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        return optimizer

In [ ]:
def progressbar(iter_, prefix="", size=60, file=sys.stdout):
    count = len(iter_)
    
    def show(j):
        x = int(size * j / count)
        file.write("{}[{}{}] {:3d}/{}\r".format(prefix, "#"*x, "."*(size-x), j, count))
        file.flush()
    show(0)
    
    loss = 0
    for i, item in enumerate(iter_):
        inputs = item.permute(0,1,4,2,3) # batch x time_steps x channel x width x height
        inputs = Variable(inputs.cuda())
        
        loss_temp = model(inputs) # batch x n_layers x nt
        loss += loss_temp
        del loss_temp
        
        show(i+1)
    file.write("\n")
    file.flush()
    
    return loss

In [ ]:
for epoch in range(100):
    optimizer = lr_scheduler(optimizer, epoch)
    
    loss = progressbar(train_loader, "step: ", 40)
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('Epoch: {}/{}, loss: {}'.format(epoch, num_epochs, loss))
    
    with pytorch.no_grad():
            torch.cuda.empty_cache()
    
    if epoch%10 == 0:
        torch.save(model.state_dict(), 'models/training_{:04d}.pt'.format(epoch))
torch.save(model.state_dict(), 'models/training.pt')